# Защита персональных данных клиентов

Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность его работы.

При этом также требуется, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [3]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [4]:
data = pd.read_csv('/Users/alexey_zalesov/Desktop/ya_prakrikum/ds/datasets/insurance.csv')
display(data.head())



,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


* 		Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* 		Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


### Первичный вывод  

Дана таблица: 5 столбцов по 5000 строчек, 4 столбца-признака и 1 столбец с целевым. 
Нужно заменить названия столбцов, проверить пропуски, дубликаты, возможно, потребуется изменить столбец "Пол" с категориального на количественный, также необходимо изменить тип данных в  колонках "Возраст" и "Зарплата". 

In [7]:
data.set_axis(['gender', 'age', 'salary', 'family_members_count', 'insurance_case'], axis='columns', inplace=True)
                

display(data.head())





,gender,age,salary,family_members_count,insurance_case
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [8]:
data['gender'].isna().sum()

0

In [9]:
data['age'].isna().sum()

0

In [10]:
data['salary'].isna().sum()

0

In [11]:
data['family_members_count'].isna().sum()

0

In [12]:
data['insurance_case'].isna().sum()

0

In [13]:
data.duplicated().sum()


153

Всего 153 дубликата, это около 3% от общего числа строк. Предлагаю удалить. 

In [14]:
data_cleared = data.drop_duplicates().reset_index(drop=True)

In [15]:
data['age'] = data['age'].astype('int')
data['salary'] = data['salary'].astype('int')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   gender                5000 non-null   int64
 1   age                   5000 non-null   int64
 2   salary                5000 non-null   int64
 3   family_members_count  5000 non-null   int64
 4   insurance_case        5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


### Вывод

Данные загружены, изучены, дубликаты удалены, нужные типы данных изменены, также изменены названия столбцов. 

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

(формула 1)  
$$
a = Xw
$$

Задача обучения:
(формула 2)  
$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:
(формула 3)  
$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 

b. Не изменится. 



**Обоснование:** 

Будем считать формулы в ячейке выше, как уже доказанные

тогда:

$$
w' = (((XP)^T)(XP)^{-1}(XP)^Ty =  
= (((P^T X^T)(XP))^{-1}(XP)^Ty =  
= ((P^T(X^T X))P)^{-1}(P^T X^T)y $$  
[так как умножение матриц ассоциативно]

$$ 
= P^{-1} ((P^T(X^T X))^{-1}(P^T X^T)y = 
$$

$$
= P^{-1} (X^T X)^{-1} (P^T)^{-1} (P^T X^T)y=
$$

$$
= P^{-1} (X^T X)^{-1} E X^T y=
$$

$$
= P^{-1} (X^T X)^{-1} X^T y =
$$

[по формуле 3:]

$$
= P^{-1} w
$$








$$
=>w'=P^{-1}w
$$  



$$
a'=XP*w'(формула1)
=>a'=XP P^{-1}w(по-доказанному) = XEw = Xw = a (формула1)
$$  




Что и требовалось доказать. 

## Алгоритм преобразования

**Алгоритм**

##### Напишем функцию, которая:
1. Создаст соответствующий Random_state
2. Создаст "случайную" матрицу с помощью numpy.random.normal()
3. Проверит ее на обратимость - а именно существует ли обратно ей
4. В случае (крайне маловероятном), что обратной матрицы не существует - вернется на шаг 2)
5. Примет исходную матрицу признаков, умножит ее на созданную обратимую
6. Вернет новую, "измененную", матрицу


## Проверка алгоритма

#### Сначала обучим модель на неизмененной матрице признаков

In [17]:
# разбиваем данные для обучения
# так как нам не требуется выбирать лучшею модель или гиперпараметры, а нужно лишь проверить, что качество 
# предсказания модели не ухудшится, то разобьем данные: 0.75 на обучения, 0.25 для тестовой.

#train, test = train_test_split 
features = data_cleared.drop('insurance_case', axis=1)
target = data_cleared['insurance_case']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25,
                                                                           random_state=12345)


In [18]:
#обучаем модель, считаем значение метрики r2

model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)

score = r2_score(target_test, predictions)

print('Метрика R2 модели на неизмененных данных составляет:', score)

Метрика R2 модели на неизмененных данных составляет: 0.42307727492147573


In [19]:
#Реализуем сам алгоритм:


def change_data(data):
    length = data.shape[1]
    np.random.seed(12345)
    multiplicator = np.random.normal(size=(length,length))
    print(multiplicator)
    try:
        np.linalg.inv(multiplicator)
    except LinAlgError:
        multiplicator = np.random.normal(size=(length,length))
    
    
    return pd.DataFrame(np.dot(data, multiplicator), index = data.index, columns=data.columns)

    
    
    
    
    

Изменим исходную матрицу признаков:

In [20]:
features_changed = change_data(features)

features_train_changed, features_test_changed, target_train, target_test = train_test_split(features_changed
                                                                                            , target,
                                                                                           test_size=0.25,
                                                                                           random_state=12345)


[[-0.20470766  0.47894334 -0.51943872 -0.5557303 ]
 [ 1.96578057  1.39340583  0.09290788  0.28174615]
 [ 0.76902257  1.24643474  1.00718936 -1.29622111]
 [ 0.27499163  0.22891288  1.35291684  0.88642934]]


In [21]:
display(features_changed.head())


#display(features_train_new.head())

,gender,age,salary,family_members_count
0,38224.186641,61881.000420,49961.234837,-64280.684721
1,29313.558467,47428.845564,38278.822267,-49242.555394
2,16206.481556,26215.538233,21153.670838,-27212.472653
3,32110.072445,52006.047856,42004.453110,-54044.730722
4,20126.326163,32571.440926,26289.724215,-33824.037786


In [22]:
model_new = LinearRegression()
model_new.fit(features_train_changed, target_train)


LinearRegression()

In [23]:
predictions_new = model_new.predict(features_test_changed)

In [24]:
score_new = r2_score(target_test , predictions_new)
print(score_new)

0.4230772749212812


In [25]:
print('Значение метрики R2 изменилось на:', (score/score_new - 1)*100, 'процентов')

Значение метрики R2 изменилось на: 4.5985437679973984e-11 процентов


### Вывод

Значение метрики R2 на измененной выборке, по сравнению с первоначальной изменилось незначительно (4.5^(10^-11)) процентов. 
Считаю алгоритм работающим, а утверждение доказанным. 